# Chapter 05-04: 객체 탐지 입문 — Detection & Segmentation

## 학습 목표
- 컴퓨터 비전의 주요 태스크(분류, 탐지, 분할)를 구분하고 각 출력 형식을 이해한다
- IoU와 mAP 평가 지표의 의미를 이해한다
- `tf.keras.applications`로 이미지 분류 추론을 구현한다
- MobileNetV2의 경량화 특성과 실시간 추론 활용을 이해한다
- TF Hub에서 사전 학습된 탐지 모델을 활용하는 방법을 파악한다

## 목차
1. [컴퓨터 비전 태스크 비교](#1)
2. [IoU와 mAP 평가 지표](#2)
3. [ResNet50으로 이미지 분류 추론](#3)
4. [MobileNetV2 경량 추론](#4)
5. [TF Hub 사전 학습 탐지 모델](#5)
6. [정리](#6)

In [ ]:
# 필수 라이브러리 임포트
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import time

# 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

# 재현성 시드
tf.random.set_seed(42)
np.random.seed(42)

print(f'TensorFlow 버전: {tf.__version__}')

## 1. 컴퓨터 비전 태스크 비교 <a id='1'></a>

## 컴퓨터 비전 태스크 비교

| 태스크 | 설명 | 출력 |
|--------|------|------|
| 분류(Classification) | 이미지 전체 클래스 판별 | 클래스 레이블 |
| 탐지(Detection) | 객체 위치 + 클래스 | Bounding Box + 레이블 |
| 분할(Segmentation) | 픽셀 단위 분류 | 마스크 이미지 |

### 태스크별 세부 유형

**분류(Classification)**
- 단일 레이블: 이미지당 하나의 클래스
- 다중 레이블: 이미지에 여러 클래스가 존재

**탐지(Detection)**
- 각 객체에 대해 `[x, y, w, h, class, confidence]` 출력
- 대표 모델: YOLO, SSD, Faster R-CNN

**분할(Segmentation)**
- 의미론적 분할(Semantic): 같은 클래스 = 같은 색
- 인스턴스 분할(Instance): 같은 클래스도 개체별로 구분
- 대표 모델: U-Net, Mask R-CNN, DeepLab

### 난이도 및 계산 비용
```
분류 < 탐지 < 의미론적 분할 < 인스턴스 분할
(쉬움/빠름)                          (어려움/느림)
```

## 2. IoU와 mAP 평가 지표 <a id='2'></a>

### IoU (Intersection over Union)

탐지 모델의 Bounding Box 정확도를 측정하는 지표:

$IoU = \frac{|A \cap B|}{|A \cup B|}$

여기서:
- $A$: 예측 Bounding Box
- $B$: 정답 Bounding Box (Ground Truth)
- $|A \cap B|$: 교집합 넓이 (겹치는 영역)
- $|A \cup B|$: 합집합 넓이

일반적으로 $IoU \geq 0.5$이면 올바른 탐지로 간주 (PASCAL VOC 기준).
$IoU \geq 0.75$ 또는 $0.95$는 더 엄격한 기준이다.

### mAP (mean Average Precision)

탐지 모델의 종합 성능 지표:

1. 각 클래스에 대해 **Precision-Recall 곡선**을 계산
2. 곡선 아래 넓이 = **AP (Average Precision)**
3. 모든 클래스의 AP 평균 = **mAP**

$mAP = \frac{1}{N} \sum_{i=1}^{N} AP_i$

### NMS (Non-Maximum Suppression)

탐지 모델은 동일 객체에 대해 여러 박스를 예측한다. NMS로 중복 제거:
1. confidence 점수 기준 내림차순 정렬
2. 가장 높은 점수의 박스 선택
3. 해당 박스와 $IoU > \text{threshold}$인 박스 제거
4. 반복

In [ ]:
def compute_iou(box1, box2):
    """
    두 Bounding Box의 IoU 계산
    
    Args:
        box1, box2: [x_min, y_min, x_max, y_max] 형식
    
    Returns:
        IoU 값 (0.0 ~ 1.0)
    """
    # 교집합 계산
    inter_x1 = max(box1[0], box2[0])
    inter_y1 = max(box1[1], box2[1])
    inter_x2 = min(box1[2], box2[2])
    inter_y2 = min(box1[3], box2[3])
    
    # 겹치지 않는 경우
    if inter_x2 <= inter_x1 or inter_y2 <= inter_y1:
        return 0.0
    
    inter_area = (inter_x2 - inter_x1) * (inter_y2 - inter_y1)
    
    # 각 박스 넓이
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    # IoU = 교집합 / 합집합
    union_area = area1 + area2 - inter_area
    return inter_area / union_area


# IoU 시각화
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 테스트 케이스: 다양한 IoU 상황
test_cases = [
    {'gt': [1, 1, 5, 5], 'pred': [3, 3, 7, 7], 'title': '부분 겹침'},
    {'gt': [1, 1, 5, 5], 'pred': [1, 1, 5, 5], 'title': '완전 일치'},
    {'gt': [1, 1, 4, 4], 'pred': [5, 5, 8, 8], 'title': '겹침 없음'},
]

for ax, case in zip(axes, test_cases):
    gt   = case['gt']
    pred = case['pred']
    iou  = compute_iou(gt, pred)
    
    # 교집합 영역
    inter_x1 = max(gt[0], pred[0])
    inter_y1 = max(gt[1], pred[1])
    inter_x2 = min(gt[2], pred[2])
    inter_y2 = min(gt[3], pred[3])
    
    ax.set_xlim(0, 9)
    ax.set_ylim(0, 9)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    
    # Ground Truth 박스 (파란색)
    gt_rect = patches.Rectangle(
        (gt[0], gt[1]), gt[2]-gt[0], gt[3]-gt[1],
        linewidth=2, edgecolor='blue', facecolor='blue', alpha=0.2,
        label='Ground Truth'
    )
    ax.add_patch(gt_rect)
    
    # 예측 박스 (빨간색)
    pred_rect = patches.Rectangle(
        (pred[0], pred[1]), pred[2]-pred[0], pred[3]-pred[1],
        linewidth=2, edgecolor='red', facecolor='red', alpha=0.2,
        label='Prediction'
    )
    ax.add_patch(pred_rect)
    
    # 교집합 영역 (초록색)
    if inter_x2 > inter_x1 and inter_y2 > inter_y1:
        inter_rect = patches.Rectangle(
            (inter_x1, inter_y1), inter_x2-inter_x1, inter_y2-inter_y1,
            linewidth=0, facecolor='green', alpha=0.5, label='교집합'
        )
        ax.add_patch(inter_rect)
    
    ax.set_title(f"{case['title']}\nIoU = {iou:.3f}", fontsize=12)
    ax.legend(loc='upper right', fontsize=8)

plt.suptitle('IoU (Intersection over Union) 시각화', fontsize=14)
plt.tight_layout()
plt.show()

# IoU 값 출력
print('IoU 계산 결과:')
for case in test_cases:
    iou = compute_iou(case['gt'], case['pred'])
    verdict = '올바른 탐지' if iou >= 0.5 else '잘못된 탐지'
    print(f"  {case['title']}: IoU = {iou:.3f} → {verdict} (임계값 0.5 기준)")

## 3. ResNet50으로 이미지 분류 추론 <a id='3'></a>

`tf.keras.applications`를 사용한 ImageNet 분류 추론:
1. `preprocess_input()`: 모델별 입력 전처리 (픽셀 정규화)
2. `decode_predictions()`: 예측 결과를 사람이 읽을 수 있는 레이블로 변환

In [ ]:
# ResNet50으로 ImageNet 분류 추론

# ResNet50 로드 (include_top=True: 분류 헤드 포함, ImageNet 1000 클래스)
resnet50 = tf.keras.applications.ResNet50(
    include_top=True,
    weights='imagenet'
)

print(f'ResNet50 입력 형태: {resnet50.input_shape}')
print(f'ResNet50 출력 형태: {resnet50.output_shape}')
print(f'총 파라미터: {resnet50.count_params():,}')

# 테스트용 이미지 생성 (실제 사용 시 실제 이미지 로드)
# np.random으로 224x224 랜덤 이미지 생성 (시연 목적)
np.random.seed(42)
dummy_image = np.random.randint(0, 256, (224, 224, 3), dtype=np.uint8)

# ResNet50 추론 파이프라인
def predict_resnet50(image_array):
    """
    ResNet50으로 이미지 분류 추론
    
    Args:
        image_array: (H, W, 3) uint8 이미지 배열
    
    Returns:
        상위 5개 예측 결과 [(클래스 ID, 클래스명, 확률), ...]
    """
    # 배치 차원 추가: (224, 224, 3) → (1, 224, 224, 3)
    img = np.expand_dims(image_array.astype(np.float32), axis=0)
    
    # ResNet50 전용 전처리 (채널별 평균 차감)
    img_preprocessed = tf.keras.applications.resnet50.preprocess_input(img)
    
    # 추론
    start_time = time.time()
    predictions = resnet50.predict(img_preprocessed, verbose=0)
    inference_time = (time.time() - start_time) * 1000
    
    # 상위 5개 결과 디코딩
    top5 = tf.keras.applications.resnet50.decode_predictions(predictions, top=5)[0]
    
    return top5, inference_time


# CIFAR-10 샘플 이미지로 추론 테스트
(_, _), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
cifar_classes = ['비행기', '자동차', '새', '고양이', '사슴',
                 '개', '개구리', '말', '배', '트럭']

# 224x224로 리사이즈
sample_img_32 = x_test[0]  # CIFAR-10 이미지 (32x32)
sample_img_224 = tf.image.resize(sample_img_32, [224, 224]).numpy().astype(np.uint8)

# 추론 실행
top5_preds, inf_time = predict_resnet50(sample_img_224)

print(f'\n추론 시간: {inf_time:.1f}ms')
print(f'실제 레이블: {cifar_classes[y_test[0][0]]}')
print('\nResNet50 예측 상위 5개:')
print(f'{'순위':<5} {'클래스 ID':<15} {'클래스명':<25} {'확률'}')
print('-' * 60)
for i, (class_id, class_name, prob) in enumerate(top5_preds):
    print(f'{i+1:<5} {class_id:<15} {class_name:<25} {prob:.4f}')

# 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.imshow(sample_img_224)
ax1.set_title(f'입력 이미지\n실제 레이블: {cifar_classes[y_test[0][0]]}')
ax1.axis('off')

class_names = [p[1] for p in top5_preds]
probs = [p[2] for p in top5_preds]
colors = ['#FF6B6B' if i == 0 else '#4ECDC4' for i in range(5)]

bars = ax2.barh(range(len(class_names)), probs, color=colors)
ax2.set_yticks(range(len(class_names)))
ax2.set_yticklabels(class_names, fontsize=9)
ax2.set_xlabel('확률')
ax2.set_title('ResNet50 예측 상위 5개')
ax2.set_xlim(0, max(probs) * 1.2)

for bar, prob in zip(bars, probs):
    ax2.text(bar.get_width() + 0.001, bar.get_y() + bar.get_height()/2,
             f'{prob:.3f}', va='center', fontsize=9)

plt.suptitle('ResNet50 ImageNet 분류 추론', fontsize=13)
plt.tight_layout()
plt.show()

## 4. MobileNetV2 경량 추론 <a id='4'></a>

MobileNetV2는 모바일 및 엣지 디바이스를 위한 경량 CNN이다.

### 핵심 기술: Depthwise Separable Convolution

일반 Conv2D를 두 단계로 분리:
1. **Depthwise Conv**: 채널별로 독립적인 공간 필터링
2. **Pointwise Conv (1×1)**: 채널 조합

파라미터 수 비교:
- 일반 Conv: $K^2 \times C_{in} \times C_{out}$
- Depthwise Separable: $K^2 \times C_{in} + C_{in} \times C_{out}$
- 비율: $\frac{1}{C_{out}} + \frac{1}{K^2}$ (K=3이면 약 8~9배 절약)

In [ ]:
# MobileNetV2와 ResNet50 추론 속도 및 정확도 비교

# MobileNetV2 로드
mobilenet_v2 = tf.keras.applications.MobileNetV2(
    include_top=True,
    weights='imagenet'
)

def predict_mobilenetv2(image_array):
    """
    MobileNetV2로 이미지 분류 추론
    
    Args:
        image_array: (H, W, 3) uint8 이미지 배열
    
    Returns:
        상위 5개 예측 결과, 추론 시간(ms)
    """
    img = np.expand_dims(image_array.astype(np.float32), axis=0)
    
    # MobileNetV2 전처리: [-1, 1] 범위로 정규화
    img_preprocessed = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    
    start_time = time.time()
    predictions = mobilenet_v2.predict(img_preprocessed, verbose=0)
    inference_time = (time.time() - start_time) * 1000
    
    top5 = tf.keras.applications.mobilenet_v2.decode_predictions(predictions, top=5)[0]
    return top5, inference_time


# 두 모델 비교 (동일 이미지)
top5_mobile, inf_time_mobile = predict_mobilenetv2(sample_img_224)

print('===== 모델 비교 =====\n')
print(f'{'항목':<25} {'ResNet50':>15} {'MobileNetV2':>15}')
print('-' * 55)
print(f'{'파라미터 수':<25} {resnet50.count_params():>15,} {mobilenet_v2.count_params():>15,}')
print(f'{'추론 시간(ms)':<25} {inf_time:>15.1f} {inf_time_mobile:>15.1f}')
print(f'{'모델 크기 비율':<25} {'1.0x':>15} {mobilenet_v2.count_params()/resnet50.count_params():>14.2f}x')
print()

# MobileNetV2 예측 결과
print('MobileNetV2 예측 상위 5개:')
print(f'{'순위':<5} {'클래스명':<30} {'확률'}')
print('-' * 45)
for i, (class_id, class_name, prob) in enumerate(top5_mobile):
    print(f'{i+1:<5} {class_name:<30} {prob:.4f}')

# 모델 크기 비교 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

models_info = {
    'ResNet50':    {'params': resnet50.count_params(),    'time': inf_time},
    'MobileNetV2': {'params': mobilenet_v2.count_params(), 'time': inf_time_mobile},
}

names = list(models_info.keys())
params = [models_info[n]['params']/1e6 for n in names]
times  = [models_info[n]['time'] for n in names]

colors = ['#FF6B6B', '#4ECDC4']

bars1 = axes[0].bar(names, params, color=colors)
axes[0].set_ylabel('파라미터 수 (백만)')
axes[0].set_title('파라미터 수 비교')
for bar, val in zip(bars1, params):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                 f'{val:.1f}M', ha='center', fontsize=11, fontweight='bold')

bars2 = axes[1].bar(names, times, color=colors)
axes[1].set_ylabel('추론 시간 (ms)')
axes[1].set_title('추론 시간 비교')
for bar, val in zip(bars2, times):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                 f'{val:.1f}ms', ha='center', fontsize=11, fontweight='bold')

plt.suptitle('ResNet50 vs MobileNetV2 비교', fontsize=13)
plt.tight_layout()
plt.show()

## 5. TF Hub 소개 및 사전 학습 탐지 모델 <a id='5'></a>

### TensorFlow Hub (TF Hub)

TF Hub는 사전 학습된 TensorFlow/Keras 모델 저장소이다.
URL 하나로 다양한 태스크의 전문 모델을 즉시 사용할 수 있다.

### TF Hub 사용 방법

```python
import tensorflow_hub as hub

# 객체 탐지 모델 로드 (SSD MobileNetV2)
detector = hub.load(
    'https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2'
)

# 추론 실행
# 입력: uint8 이미지 텐서 (1, H, W, 3)
img_tensor = tf.expand_dims(tf.cast(image, tf.uint8), axis=0)
result = detector(img_tensor)

# 결과 파싱
boxes   = result['detection_boxes']       # (1, 100, 4) - [y1, x1, y2, x2] 정규화 좌표
classes = result['detection_classes']     # (1, 100) - 클래스 ID
scores  = result['detection_scores']      # (1, 100) - 신뢰도 점수

# 신뢰도 > 0.5인 탐지만 시각화
threshold = 0.5
for i in range(len(scores[0])):
    if scores[0][i] >= threshold:
        box = boxes[0][i].numpy()  # [y1, x1, y2, x2]
        print(f'클래스: {classes[0][i]:.0f}, 점수: {scores[0][i]:.3f}')
```

### TF Hub 주요 탐지 모델

| 모델 | 속도 | 정확도 | 용도 |
|------|------|--------|------|
| SSD MobileNetV2 | 매우 빠름 | 중간 | 실시간 탐지, 모바일 |
| SSD ResNet50 | 중간 | 높음 | 균형적 성능 |
| Faster R-CNN ResNet101 | 느림 | 매우 높음 | 오프라인 고정밀 탐지 |
| EfficientDet D7 | 매우 느림 | 최고 | 연구, 벤치마크 |

### TF Hub 설치 및 기본 사용

```bash
# TF Hub 설치
pip install tensorflow-hub
```

TF Hub URL 형식: `https://tfhub.dev/<publisher>/<model-name>/<version>`

탐지 모델 결과에는 일반적으로:
- `detection_boxes`: 정규화된 [y_min, x_min, y_max, x_max] 좌표
- `detection_classes`: COCO 클래스 ID
- `detection_scores`: 신뢰도 점수
- `num_detections`: 탐지된 객체 수

COCO 데이터셋은 80개 클래스 (사람, 자동차, 동물 등)로 학습된 모델이 많다.

In [ ]:
# TF Hub 탐지 결과 파싱 및 시각화 함수 (실제 사용 시 참고)

# COCO 클래스 레이블 (일부)
COCO_LABELS = {
    1: '사람', 2: '자전거', 3: '자동차', 4: '오토바이', 5: '비행기',
    6: '버스', 7: '기차', 8: '트럭', 9: '배', 10: '신호등',
    16: '새', 17: '고양이', 18: '개', 19: '말', 20: '양',
    21: '소', 44: '병', 47: '컵', 51: '그릇', 67: '핸드폰',
}

def visualize_detections(image, boxes, classes, scores, threshold=0.5,
                          label_map=None):
    """
    TF Hub 탐지 결과 시각화 함수
    
    Args:
        image: (H, W, 3) 이미지 배열
        boxes: (N, 4) Bounding Box [y1, x1, y2, x2] 정규화 좌표
        classes: (N,) 클래스 ID
        scores: (N,) 신뢰도 점수
        threshold: 표시 임계값
        label_map: {클래스 ID → 클래스명} 딕셔너리
    """
    h, w = image.shape[:2]
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    ax.imshow(image)
    
    n_detections = 0
    for i in range(len(scores)):
        if scores[i] < threshold:
            continue
        
        # 정규화 좌표 → 픽셀 좌표 변환
        y1, x1, y2, x2 = boxes[i]
        x1_px, y1_px = int(x1 * w), int(y1 * h)
        x2_px, y2_px = int(x2 * w), int(y2 * h)
        
        # Bounding Box 그리기
        rect = patches.Rectangle(
            (x1_px, y1_px), x2_px - x1_px, y2_px - y1_px,
            linewidth=2, edgecolor='red', facecolor='none'
        )
        ax.add_patch(rect)
        
        # 레이블 및 신뢰도 표시
        class_id = int(classes[i])
        label = label_map.get(class_id, str(class_id)) if label_map else str(class_id)
        ax.text(x1_px, y1_px - 5, f'{label}: {scores[i]:.2f}',
                fontsize=10, color='red',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
        
        n_detections += 1
    
    ax.set_title(f'탐지 결과 (임계값: {threshold}, 탐지된 객체: {n_detections}개)')
    ax.axis('off')
    plt.tight_layout()
    plt.show()
    
    return n_detections


# 탐지 결과 파싱 예시 (더미 데이터로 시연)
print('탐지 모델 결과 파싱 예시 (더미 데이터):')
print()

# 더미 탐지 결과 생성
dummy_boxes   = np.array([[0.1, 0.1, 0.5, 0.4],   # 사람
                            [0.5, 0.3, 0.9, 0.8],   # 자동차
                            [0.0, 0.0, 0.2, 0.3]])  # 저신뢰도
dummy_classes = np.array([1, 3, 2])                 # 사람, 자동차, 자전거
dummy_scores  = np.array([0.92, 0.87, 0.31])        # 신뢰도

print(f'{'인덱스':<8} {'클래스':<10} {'신뢰도':<10} {'좌표 [y1,x1,y2,x2]':<25} {'표시 여부'}')
print('-' * 70)
threshold = 0.5
for i, (box, cls, score) in enumerate(zip(dummy_boxes, dummy_classes, dummy_scores)):
    label = COCO_LABELS.get(int(cls), str(int(cls)))
    show = '표시' if score >= threshold else '무시'
    print(f'{i:<8} {label:<10} {score:<10.3f} {str(box.tolist()):<25} {show}')

print(f'\n→ 임계값 {threshold} 기준: {sum(dummy_scores >= threshold)}개 탐지 표시')

## 6. 정리 <a id='6'></a>

### 컴퓨터 비전 태스크 요약

| 태스크 | 입력 | 출력 | 평가 지표 | 대표 모델 |
|--------|------|------|----------|-----------|
| 분류 | 이미지 | 클래스 확률 | Accuracy, Top-5 Acc | ResNet, EfficientNet |
| 탐지 | 이미지 | Box + 클래스 | mAP@IoU | YOLO, SSD, Faster RCNN |
| 분할 | 이미지 | 픽셀 마스크 | mIoU | U-Net, DeepLab, SegFormer |

### 핵심 평가 지표
- $IoU = \frac{|A \cap B|}{|A \cup B|}$: Bounding Box 겹침 정도
- $mAP = \frac{1}{N} \sum_{i=1}^{N} AP_i$: 탐지 종합 성능

### 모델 선택 가이드
- **정확도 우선**: ResNet50, EfficientNetB4 이상
- **속도 우선 (모바일/엣지)**: MobileNetV2, **EfficientNetV2B0** *(구버전 EfficientNetB0 대체; 2026-02-25 기준)*
- **탐지 (실시간)**: YOLOv8, SSD MobileNetV2
- **탐지 (고정밀)**: Faster R-CNN ResNet101, EfficientDet

### TF Hub 활용
- `pip install tensorflow-hub`
- URL 하나로 전문화된 사전 학습 모델 즉시 사용
- 탐지, 분류, 분할, 임베딩 등 다양한 태스크 지원

### 다음 챕터 예고
**Chapter 05 실습**: CIFAR-10 분류기 구현 및 Flowers 전이학습 실습을 통해
학습한 내용을 종합적으로 적용한다.